In [245]:
import pandas as pd
import opendatasets as od


In [246]:
# od.download("https://www.kaggle.com/competitions/spaceship-titanic")

In [247]:
raw_data=pd.read_csv('spaceship-titanic/train.csv')
# raw_data=raw_data[~raw_data['Cabin'].isnull()]



In [248]:
ytrain=raw_data['Transported']

In [249]:


def clean_dt(raw_data):
    return (raw_data
    .drop(columns=['Name','PassengerId'])
    .assign(
        RoomService=raw_data.RoomService.fillna(0).replace('nan',0),
        CryoSleep=raw_data.CryoSleep.fillna(False),
        VIP=raw_data.VIP.fillna(False).replace('nan',0),
        FoodCourt=raw_data.FoodCourt.fillna(0).replace('nan',0),
        ShoppingMall=raw_data.ShoppingMall.fillna(0).replace('nan',0),
        Spa=raw_data.Spa.fillna(0).replace('nan',0),
        VRDeck=raw_data.VRDeck.fillna(0).replace('nan',0),
        Cabin=raw_data.Cabin.str.split('/',expand=True)[2]=='S',
        Age=raw_data.Age.fillna(raw_data.Age.median())
        )
     
    )



In [250]:
 # encoding boolean labels
from sklearn import preprocessing

def transform_step(raw_data):
    label_encoder=preprocessing.LabelEncoder()
    raw_data.CryoSleep=label_encoder.fit_transform(raw_data.CryoSleep).astype(int)
    raw_data.Cabin=label_encoder.fit_transform(raw_data.Cabin).astype(int)
    raw_data.VIP=label_encoder.fit_transform(raw_data.VIP)
    raw_data=pd.get_dummies(raw_data,columns=['Destination','HomePlanet'])
    return raw_data

In [251]:
res_encoder=preprocessing.LabelEncoder()

ytrain=res_encoder.fit_transform(ytrain)

In [252]:
  
Xtrain=clean_dt(raw_data)
Xtrain=transform_step(Xtrain)
Xtrain.drop(columns=['Transported'],inplace=True)   



In [253]:
raw_test=pd.read_csv('spaceship-titanic/test.csv')
# raw_test=raw_test[~raw_test['Cabin'].isnull()]

Xtest=clean_dt(raw_test)
Xtest=transform_step(Xtest)

In [254]:
# Port=Xtrain.Cabin.str.split('/',expand=True)[2]=="P"
# new_cab=new_cab[2]
# S_Cabin= Xtrain.loc[].split()== "S"
# Xtrain=Xtrain[~Xtrain['Cabin'].isnull()]

In [255]:
Xtrain.dtypes

CryoSleep                      int32
Cabin                          int32
Age                          float64
VIP                            int64
RoomService                  float64
FoodCourt                    float64
ShoppingMall                 float64
Spa                          float64
VRDeck                       float64
Destination_55 Cancri e        uint8
Destination_PSO J318.5-22      uint8
Destination_TRAPPIST-1e        uint8
HomePlanet_Earth               uint8
HomePlanet_Europa              uint8
HomePlanet_Mars                uint8
dtype: object

In [256]:
(Xtrain
 .assign(Transported=ytrain==True)
 .corr(method='spearman')
 .style
 .background_gradient(cmap='RdBu',vmax=1,vmin=-1)
 .set_sticky(axis='index')
)

,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Transported
CryoSleep,1.000000,0.016861,-0.073039,-0.078281,-0.508773,-0.522937,-0.504369,-0.538649,-0.517701,0.065589,0.087764,-0.103213,-0.109133,0.093395,0.032715,0.460132
Cabin,0.016861,1.000000,0.009555,-0.011017,-0.020457,0.008792,-0.011945,0.002041,-0.009594,0.010285,-0.006098,-0.005771,-0.015226,0.034147,-0.019279,0.101455
Age,-0.073039,0.009555,1.000000,0.103031,0.121018,0.200438,0.099445,0.191171,0.176715,0.022422,-0.038165,-0.001028,-0.226389,0.239897,0.025840,-0.069435
VIP,-0.078281,-0.011017,0.103031,1.000000,0.051972,0.111360,0.033293,0.086885,0.094900,0.045167,-0.000592,-0.035309,-0.162341,0.147008,0.043523,-0.037261
RoomService,-0.508773,-0.020457,0.121018,0.051972,1.000000,0.181157,0.430106,0.239521,0.180872,-0.072356,-0.051460,0.090983,-0.025766,-0.200659,0.250059,-0.363899
FoodCourt,-0.522937,0.008792,0.200438,0.111360,0.181157,1.000000,0.185407,0.470270,0.497026,0.055925,-0.062319,-0.010725,-0.057518,0.267833,-0.214710,-0.186388
ShoppingMall,-0.504369,-0.011945,0.099445,0.033293,0.430106,0.185407,1.000000,0.253359,0.187538,-0.081787,-0.028479,0.086086,0.024819,-0.190941,0.178022,-0.227018
Spa,-0.538649,0.002041,0.191171,0.086885,0.239521,0.470270,0.253359,1.000000,0.437234,0.017912,-0.069637,0.024708,-0.060307,0.170405,-0.106003,-0.366099
VRDeck,-0.517701,-0.009594,0.176715,0.094900,0.180872,0.497026,0.187538,0.437234,1.000000,0.037693,-0.050928,-0.005362,-0.008764,0.201273,-0.202459,-0.347008
Destination_55 Cancri e,0.065589,0.010285,0.022422,0.045167,-0.072356,0.055925,-0.081787,0.017912,0.037693,1.000000,-0.162240,-0.745664,-0.149534,0.293517,-0.120996,0.108722


It seems that the persons who have billed luxury amenities have a negative relationship with being transported. In simple words, the people who have bills are less likely to be transported.

In [257]:
# prediction
import xgboost as xgb

random_for=xgb.XGBRFClassifier()
random_for.fit(Xtrain,ytrain)
res=random_for.predict(Xtest)

In [258]:
decoded_res=res_encoder.inverse_transform(res)

In [259]:
# submission
result=pd.DataFrame({'PassengerId':raw_test.PassengerId,'Transported':decoded_res})

In [260]:
result

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,False


In [261]:
result.to_csv('sub1.csv',index=False)


In [262]:
stump_dt=xgb.XGBClassifier()
stump_dt.fit(Xtrain,ytrain)
res2=stump_dt.predict(Xtest)

In [ ]:
res2=pd.DataFrame({'PassengerId':raw_test.PassengerId,'Transported':res_encoder.inverse_transform(res)})

res2.to_csv('res2.csv',index=False)